# sensPy Tutorial

This notebook demonstrates the main features of sensPy, a Python library for Thurstonian models in sensory discrimination analysis.

## Contents
1. Basic Discrimination Analysis
2. Psychometric Functions
3. Power Analysis
4. Advanced Models
5. ROC Curves and Signal Detection
6. Interactive Visualizations

In [ ]:
import numpy as np
import senspy as sp

print(f"sensPy version: {sp.__version__}")

## 1. Basic Discrimination Analysis

The `discrim()` function analyzes data from forced-choice discrimination tests.

In [ ]:
# Triangle test: 80 correct out of 100 trials
result = sp.discrim(correct=80, total=100, method="triangle")

print(f"D-prime: {result.d_prime:.3f}")
print(f"Standard error: {result.se_d_prime:.3f}")
print(f"Proportion correct (Pc): {result.pc:.3f}")
print(f"Proportion discriminating (Pd): {result.pd:.3f}")
print(f"P-value: {result.p_value:.4f}")

In [ ]:
# Compare different protocols with the same data
protocols = ["triangle", "duotrio", "twoAFC", "threeAFC", "tetrad"]

print(f"{'Protocol':<12} {'d-prime':>8} {'SE':>8} {'Pc':>8}")
print("-" * 40)

for protocol in protocols:
    r = sp.discrim(correct=80, total=100, method=protocol)
    print(f"{protocol:<12} {r.d_prime:>8.3f} {r.se_d_prime:>8.3f} {r.pc:>8.3f}")

## 2. Psychometric Functions

Psychometric functions describe the relationship between d-prime and proportion correct.

In [ ]:
# Convert d-prime to proportion correct
d_prime = 1.5

for method in ["triangle", "twoAFC", "tetrad"]:
    pc = sp.psy_fun(d_prime=d_prime, method=method)
    print(f"{method}: Pc = {pc:.3f}")

In [ ]:
# Visualize psychometric functions
fig = sp.plot_psychometric_comparison()
fig.show()

In [ ]:
# Single protocol with guessing probability line
fig = sp.plot_psychometric(method="triangle")
fig.show()

## 3. Power Analysis

Plan your study by calculating power and required sample sizes.

In [ ]:
# Calculate power for a triangle test
power = sp.discrim_power(
    d_prime=1.0,
    n=100,
    method="triangle",
    alpha=0.05
)
print(f"Power with N=100, d'=1.0: {power:.1%}")

In [ ]:
# Calculate required sample size for 80% power
n_required = sp.discrim_sample_size(
    d_prime=1.0,
    method="triangle",
    power=0.80,
    alpha=0.05
)
print(f"Required N for 80% power: {n_required}")

In [ ]:
# Compare sample sizes across protocols
print(f"{'Protocol':<12} {'N (80% power)':>15}")
print("-" * 30)

for protocol in protocols:
    n = sp.discrim_sample_size(d_prime=1.0, method=protocol, power=0.8)
    print(f"{protocol:<12} {n:>15}")

In [ ]:
# Plot power curve
d_values = np.linspace(0, 3, 50)
powers = [sp.discrim_power(d_prime=d, n=100, method="triangle") for d in d_values]

fig = sp.plot_power_curve(d_values, powers, target_power=0.8)
fig.update_layout(title="Power Curve for Triangle Test (N=100)")
fig.show()

## 4. Advanced Models

### Beta-Binomial Model

For replicated discrimination data with potential overdispersion.

In [ ]:
# Replicated triangle test data: [correct, total] for each assessor
data = np.array([
    [6, 10],   # Assessor 1: 6/10 correct
    [7, 10],   # Assessor 2: 7/10 correct
    [5, 10],   # Assessor 3: 5/10 correct
    [8, 10],   # Assessor 4: 8/10 correct
    [6, 10],   # Assessor 5: 6/10 correct
    [9, 10],   # Assessor 6: 9/10 correct
])

result = sp.betabin(data, method="triangle")
print(f"D-prime: {result.d_prime:.3f}")
print(f"Gamma (overdispersion): {result.gamma:.3f}")

### D-Prime Comparison

Compare d-prime across multiple groups.

In [ ]:
# Three products tested with triangle test
correct = [75, 60, 80]
total = [100, 100, 100]
groups = ["Product A", "Product B", "Product C"]

result = sp.dprime_compare(
    correct=correct,
    total=total,
    groups=groups,
    protocol="triangle"
)

print(f"Chi-squared: {result.statistic:.2f}")
print(f"P-value: {result.p_value:.4f}")
print(f"\nGroup estimates:")
for g, d, se in zip(groups, result.d_primes, result.se_d_primes):
    print(f"  {g}: d' = {d:.3f} (SE = {se:.3f})")

In [ ]:
# Post-hoc pairwise comparisons
posthoc_result = sp.posthoc(
    correct=correct,
    total=total,
    groups=groups,
    protocol="triangle"
)

print("Pairwise comparisons:")
for comp, pval in posthoc_result.p_values.items():
    sig = "*" if pval < 0.05 else ""
    print(f"  {comp}: p = {pval:.4f} {sig}")

print(f"\nCompact Letter Display: {posthoc_result.letters}")

## 5. ROC Curves and Signal Detection

Receiver Operating Characteristic analysis for discrimination data.

In [ ]:
# Compute AUC from d-prime
d = 1.5
auc_result = sp.auc(d=d, se_d=0.2)

print(f"D-prime: {d}")
print(f"AUC: {auc_result.value:.3f}")
print(f"95% CI: [{auc_result.lower:.3f}, {auc_result.upper:.3f}]")

In [ ]:
# Plot ROC curve with confidence band
fig = sp.plot_roc(d_prime=1.5, se_d=0.2)
fig.show()

In [ ]:
# Visualize signal detection distributions
fig = sp.plot_sdt_distributions(d_prime=1.5)
fig.show()

## 6. A-Not-A Protocol

Signal detection analysis for A-Not-A discrimination.

In [ ]:
# A-Not-A data:
# x1=80 A samples correctly identified out of n1=100
# x2=70 Not-A samples correctly identified out of n2=100
result = sp.anota(x1=80, n1=100, x2=70, n2=100)

print(f"D-prime: {result.d_prime:.3f}")
print(f"SE: {result.se_d_prime:.3f}")
print(f"Hit rate: {result.hit_rate:.2%}")
print(f"False alarm rate: {result.false_alarm_rate:.2%}")
print(f"P-value: {result.p_value:.4f}")

## Summary

sensPy provides a complete toolkit for sensory discrimination analysis:

- **Basic analysis**: `discrim()` for forced-choice tests
- **Psychometric functions**: `psy_fun()`, `psy_inv()`, link functions
- **Power analysis**: `discrim_power()`, `discrim_sample_size()`
- **Advanced models**: `betabin()`, `twoac()`, `samediff()`, `dod()`
- **Group comparisons**: `dprime_compare()`, `posthoc()`
- **ROC analysis**: `roc()`, `auc()`, `sdt()`
- **Interactive plots**: `plot_roc()`, `plot_psychometric()`, etc.

For more details, see the [documentation](https://aigorahub.github.io/sensPy).